# This notebook is to help automatically import csv file to hive

In [1]:
%%sh
mkdir -p /v3io/${V3IO_HOME}/examples
mkdir -p /v3io/${V3IO_HOME}/examples/csvc

# Download a sample stocks file from Iguazio demo bucket in S3
curl -L "https://s3.wasabisys.com/iguazio/data/stocks/2018-03-26_BINS_XETR08.csv" > /v3io/${V3IO_HOME}/examples/demo.csv
curl -L "https://s3.wasabisys.com/iguazio/data/stocks/2018-03-26_BINS_XETR08.csv" > /v3io/${V3IO_HOME}/examples/csvs/example1.csv
curl -L "https://s3.wasabisys.com/iguazio/data/stocks/2018-03-26_BINS_XETR08.csv" > /v3io/${V3IO_HOME}/examples/csvs/example2.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  861k  100  861k    0     0   380k      0  0:00:02  0:00:02 --:--:--  380k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  861k  100  861k    0     0   356k      0  0:00:02  0:00:02 --:--:--  356k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  861k  100  861k    0     0   306k      0  0:00:02  0:00:02 --:--:--  306k


Below is import of all needed dependencies. And in this sell you should pass path where parquet files located. 

In [2]:
import os
HOME = os.getenv('V3IO_HOME')
V3IO_HOME_URL=os.getenv('V3IO_HOME_URL')

Here is creating of spark context with hive support.

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Import parquet schema to hive").config("hive.metastore.uris", "thrift://trino-hive:9083").enableHiveSupport().getOrCreate()

Define function below for getting sql script needed for creating table in hive using dataframe types as columns to table

In [4]:
def getCreateTableScriptCSV(databaseName, tableName, path, df):
    cols = df.dtypes
    createScript = "CREATE EXTERNAL TABLE IF NOT EXISTS " + databaseName + "." + tableName + "("
    colArray = []
    for colName, colType in cols:
        colArray.append(colName.replace(" ", "_") + " " + colType)
    createColsScript =   ", ".join(colArray )
    
    script = createScript + createColsScript + ") ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LOCATION '" + path + "' TBLPROPERTIES('skip.header.line.count'='1') "
    print (script)
    return script

In [5]:
#define main function for creating table where arqument 'path' is path to parquet files 
def createTableCSV(databaseName, tableName, path): 
    df = spark.read.format("csv").option("header", "true").option("inferschema","true").load(path)
    sqlScript = getCreateTableScriptCSV(databaseName, tableName, path, df)
    spark.sql(sqlScript)

## One file example

In [6]:
# Set path where the csv file located.
my_csv_file_path = os.path.join(V3IO_HOME_URL, 'examples/demo.csv')
createTableCSV("default","csv_table",my_csv_file_path)

CREATE EXTERNAL TABLE IF NOT EXISTS default.csv_table(ISIN string, Mnemonic string, SecurityDesc string, SecurityType string, Currency string, SecurityID int, Date string, Time string, StartPrice double, MaxPrice double, MinPrice double, EndPrice double, TradedVolume int, NumberOfTrades int) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LOCATION 'v3io://users/dani/examples/demo.csv' TBLPROPERTIES('skip.header.line.count'='1') 


## One folder example for spark output job

In [7]:
# Set path where parquet folder with csv files inside located.
folder_path = os.path.join(V3IO_HOME_URL, 'examples/csvs/')
createTableCSV("default","table_from_dir2",folder_path)

CREATE EXTERNAL TABLE IF NOT EXISTS default.table_from_dir2(ISIN string, Mnemonic string, SecurityDesc string, SecurityType string, Currency string, SecurityID string, Date string, Time string, StartPrice string, MaxPrice string, MinPrice string, EndPrice string, TradedVolume string, NumberOfTrades int) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LOCATION 'v3io://users/dani/examples/csvs/' TBLPROPERTIES('skip.header.line.count'='1') 


# Multiple files and folders example

Write here name of database and path to folder where all csv's files (or folders with them) located. Database should be created.
In this cell code goes over all files and dirs in provided path and using them for creating table.
File should be ended with .csv format and should be "," seperated.
Directory (in which stored csv files) should be started with "."
Name of directory or file will be name of table.

In [8]:
databaseName = "default"
filepath = os.path.join('/v3io', HOME, 'examples/csvs')

for fileOrDir in os.listdir(filepath):
    if fileOrDir.endswith(".csv") :
        createTableCSV(databaseName, fileOrDir.split(".csv")[0], filepath.replace("/v3io/", "v3io://", 1) + "/" + fileOrDir)
    elif not fileOrDir.startswith(".") :
        createTableCSV(databaseName, fileOrDir, filepath.replace("/v3io/", "v3io://", 1) + "/" + fileOrDir + "/*")



CREATE EXTERNAL TABLE IF NOT EXISTS default.demo1(id int, street string, city string, zip int, state string, beds int, baths int, sq__ft int, type string, sale_date string, price int, latitude double, longitude double) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LOCATION 'v3io://users/dani/examples/csvs/demo1.csv' TBLPROPERTIES('skip.header.line.count'='1') 
CREATE EXTERNAL TABLE IF NOT EXISTS default.demo2(id int, street string, city string, zip int, state string, beds int, baths int, sq__ft int, type string, sale_date string, price int, latitude double, longitude double) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LOCATION 'v3io://users/dani/examples/csvs/demo2.csv' TBLPROPERTIES('skip.header.line.count'='1') 
CREATE EXTERNAL TABLE IF NOT EXISTS default.example1(ISIN string, Mnemonic string, SecurityDesc string, SecurityType string, Currency string, SecurityID int, Date string, Time string, StartPrice double, MaxPrice double, MinPrice double, EndPrice double, TradedVolume int, Nu

# Test how it works

In [9]:
# test how the tables were saved
#spark.sql("drop database test CASCADE")
spark.sql("show databases").show()
spark.sql("show tables in " + databaseName).show()

+---------+
|namespace|
+---------+
|  default|
+---------+

+---------+---------------+-----------+
|namespace|      tableName|isTemporary|
+---------+---------------+-----------+
|  default|      csv_table|      false|
|  default|          demo1|      false|
|  default|          demo2|      false|
|  default|       example1|      false|
|  default|       example2|      false|
|  default|table_from_dir2|      false|
+---------+---------------+-----------+



In [10]:
# test how saving to table works
tableName = "example1"
spark.sql("select * from " + databaseName + "." + tableName)

DataFrame[ISIN: string, Mnemonic: string, SecurityDesc: string, SecurityType: string, Currency: string, SecurityID: int, Date: string, Time: string, StartPrice: double, MaxPrice: double, MinPrice: double, EndPrice: double, TradedVolume: int, NumberOfTrades: int]

In [11]:
spark.sql("drop table " + databaseName + ".example1")

DataFrame[]